<a href="https://colab.research.google.com/github/yutan0565/colab_git/blob/main/2_week_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
vgg = VGG16(weights = 'imagenet')
resnet = ResNet50(weights = 'imagenet')
efficientnet = EfficientNetB0(weights = 'imagenet')

In [ ]:
print(vgg.summary())
"""
 [(None, 224, 224, 3)]
 Total params: 138,357,544
Trainable params: 138,357,544
Non-trainable params: 0
"""
print(resnet.summary())
"""
[(None, 224, 224, 3  0)]
Total params: 25,636,712
Trainable params: 25,583,592
Non-trainable params: 53,120
"""
print(efficientnet.summary())
"""
[(None, 224, 224, 3  0)]
Total params: 5,330,571
Trainable params: 5,288,548
Non-trainable params: 42,023
"""

In [ ]:
# Denselayer 부분만 양자화 해주기
!pip install tensorflow_model_optimization
import tensorflow as tf
import tensorflow_model_optimization as tfmot

def apply_quantization_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

annotated_model = tf.keras.models.clone_model(
    vgg,
    clone_function=apply_quantization_to_dense,
)

quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()


In [ ]:
# 전체 양자화 해주기
quant_aware_model_all = tfmot.quantization.keras.quantize_model(vgg)
quant_aware_model_all.summary()

In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!mkdir tiny_imagenet

In [10]:
import glob
import os
import shutil

src_path = "./tiny_imagenet/"
train_dst_path = "./train/"
test_dst_path = "./test/"
split_ratio = 0.9



def check_folder_and_make(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def main():
    check_folder_and_make(train_dst_path)
    check_folder_and_make(test_dst_path)
    folders = os.listdir(src_path)
    for folder in folders:
        check_folder_and_make(train_dst_path+folder)
        check_folder_and_make(test_dst_path+folder)
        img_paths = glob.glob(src_path+folder+"/images/*.JPEG")

        img_len = len(img_paths)
        train_index = int(img_len * split_ratio)

        train_set = img_paths[:train_index]
        test_set = img_paths[train_index:]

        for train in train_set:
            shutil.copy2(train, train_dst_path+folder)

        for test in test_set:
            shutil.copy2(test, test_dst_path+folder)

if __name__ == "__main__":
    main()

In [ ]:
if __name__ == "__main__":
    im = cv2.resize(cv2.imread('cat.jpg'), (224, 224)).astype(np.float32)
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)

    # Test pretrained model
    model = VGG_16('vgg16_weights.h5')
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')
    out = model.predict(im)
    print np.argmax(out)